# Xarray Raster Alignment Example

Same CRS, but offset Affines

## No RasterIndex

In [1]:
import xarray as xr
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'

In [2]:
# Same acquisition data 
# same CRS but different resolution & therefore different affines
b4 = 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/36/M/YB/2020/8/S2A_36MYB_20200814_0_L2A/B04.tif'
b5 = 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/36/M/YB/2020/8/S2A_36MYB_20200814_0_L2A/B05.tif'

In [3]:
red = xr.open_dataarray(b4, engine='rasterio', chunks='auto').squeeze()
red

<xarray.DataArray 'band_data' (y: 10980, x: 10980)> Size: 482MB
dask.array<getitem, shape=(10980, 10980), dtype=float32, chunksize=(5120, 5120), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 88kB 7e+05 7e+05 7e+05 ... 8.097e+05 8.098e+05
  * y            (y) float64 88kB 9.7e+06 9.7e+06 9.7e+06 ... 9.59e+06 9.59e+06
    spatial_ref  int64 8B ...
Attributes:
    OVR_RESAMPLING_ALG:  AVERAGE
    AREA_OR_POINT:       Area

In [4]:
red_edge = xr.open_dataarray(b5, engine='rasterio', chunks='auto').squeeze()
red_edge

<xarray.DataArray 'band_data' (y: 5490, x: 5490)> Size: 121MB
dask.array<getitem, shape=(5490, 5490), dtype=float32, chunksize=(5490, 5490), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 44kB 7e+05 7e+05 7e+05 ... 8.097e+05 8.098e+05
  * y            (y) float64 44kB 9.7e+06 9.7e+06 9.7e+06 ... 9.59e+06 9.59e+06
    spatial_ref  int64 8B ...
Attributes:
    OVR_RESAMPLING_ALG:  AVERAGE
    AREA_OR_POINT:       Area

In [5]:
# Both same UTM Zone
red.rio.crs.to_epsg() 

32736

In [6]:
red_edge.rio.crs == red.rio.crs

True

In [7]:
red_edge.rio.transform()

Affine(20.0, 0.0, 699960.0,
       0.0, -20.0, 9700000.0)

In [8]:
red.rio.transform()

Affine(10.0, 0.0, 699960.0,
       0.0, -10.0, 9700000.0)

### Raise warning here?

In [9]:
# Naive pixel band ratio (might expect automatic coarsening here)
# Could be good to raise warning if *affine* not the same
red / red_edge

<xarray.DataArray 'band_data' (y: 0, x: 0)> Size: 0B
dask.array<truediv, shape=(0, 0), dtype=float32, chunksize=(0, 0), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 0B 
  * y            (y) float64 0B 
    band         int64 8B 1
    spatial_ref  int64 8B 0

In [11]:
# An aside / special case but COGs have pre-computed overviews which 
# often match the lower resolution coordinates of other bands
# it could be neat if the engine were 'aware' of these !
red_ovr2 = xr.open_dataarray(b4,
                               engine='rasterio', 
                               open_kwargs=dict(overview_level=0), 
                               chunks='auto').squeeze()

In [12]:
red_ovr2 + red_edge

<xarray.DataArray 'band_data' (y: 5490, x: 5490)> Size: 121MB
dask.array<add, shape=(5490, 5490), dtype=float32, chunksize=(5490, 5490), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 44kB 7e+05 7e+05 7e+05 ... 8.097e+05 8.098e+05
  * y            (y) float64 44kB 9.7e+06 9.7e+06 9.7e+06 ... 9.59e+06 9.59e+06
    spatial_ref  int64 8B 0

## With RasterIndex !

In [13]:
# TODO: rasterix pyproject
import sys
sys.path.append("..")
from rasterix import RasterIndex

In [14]:
def set_raster_index(obj):
    """Return a new DataArray or Dataset with a RasterIndex."""
    x_dim = obj.rio.x_dim
    y_dim = obj.rio.y_dim

    index = RasterIndex.from_transform(
        obj.rio.transform(),
        obj.sizes[x_dim],
        obj.sizes[y_dim],
        x_dim=x_dim,
        y_dim=y_dim,
    )

    # drop-in replacement of explicit x/y coordinates for now
    coords = xr.Coordinates.from_xindex(index)
    return obj.assign_coords(coords)


red_i = set_raster_index(red)
red_edge_i = set_raster_index(red_edge)

In [16]:
red_i / red_edge_i

NotImplementedError: RasterIndex
'x':
    <rasterix.raster_index.AxisAffineTransformIndex object at 0x7fdf81e95e90>
'y':
    <rasterix.raster_index.AxisAffineTransformIndex object at 0x7fdf78aca850> doesn't support alignment with inner/outer join method